In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os

labels = {
    0: 'bird',
    1: 'eight',
    2: 'falcon',
    3: 'five',
    4: 'four',
    5: 'nine',
    6: 'one',
    7: 'seven',
    8: 'six',
    9: 'snake',
    10: 'three',
    11: 'two',
    12: 'zero'}

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="models_data_v2_12_labels/my 97.84 submitted/tflite_non_stream/non_stream.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

audio, _ = tf.audio.decode_wav(tf.io.read_file('s1_test_release/s1_test_0003.wav'))  # snake
waveform = tf.expand_dims(tf.squeeze(audio, axis=-1), axis=-2)
# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = waveform
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
softmax_output = tf.nn.softmax(output_data)[0].numpy()
print(softmax_output)
assert len(softmax_output) == len(labels)
print(labels[np.argmax(softmax_output)])

predicted_labels, softmax_outputs, filenames = [], [], []
test_files = tf.io.gfile.glob('s1_test_release_normalized/*')
print('Test set size', len(test_files))
i = 0
print('Predicting...')
for audio in test_files:
    audio, _ = tf.audio.decode_wav(tf.io.read_file(audio))
    waveform = tf.expand_dims(tf.squeeze(audio, axis=-1), axis=-2)
    filenames.append(os.path.basename(test_files[i].replace('_normalized', '')))
    interpreter.set_tensor(input_details[0]['index'], waveform)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    softmax_output = tf.nn.softmax(output_data)[0].numpy()
    softmax_outputs.append(softmax_output)
    predicted_labels.append(labels[np.argmax(softmax_output)])
    i+=1

df = pd.DataFrame(list(zip(filenames, predicted_labels)))
df.to_csv('challenge_2_team_Tensor is not flowing.csv', index=False, header=False)  # tested submission file format passed
print(df)

df = pd.DataFrame(list(zip(filenames, softmax_outputs)))  # for ensemble
df.to_csv('softmax_outputs2.csv', index=False, header=False)
print(df)

[0.00908693 0.00699757 0.00455129 0.00456361 0.00459282 0.00384177
 0.00617683 0.00531144 0.01028626 0.9319114  0.00497585 0.00371441
 0.00398984]
snake
Test set size 649
Predicting...
                    0       1
0    s1_test_0000.wav   seven
1    s1_test_0001.wav  falcon
2    s1_test_0002.wav    four
3    s1_test_0003.wav   snake
4    s1_test_0004.wav    four
..                ...     ...
644  s1_test_0644.wav   seven
645  s1_test_0645.wav     one
646  s1_test_0646.wav    five
647  s1_test_0647.wav  falcon
648  s1_test_0648.wav     two

[649 rows x 2 columns]
                    0                                                  1
0    s1_test_0000.wav  [0.006725632, 0.008208475, 0.007295013, 0.0070...
1    s1_test_0001.wav  [0.014264489, 0.006689504, 0.67721355, 0.00483...
2    s1_test_0002.wav  [0.018799435, 0.011402309, 0.38425818, 0.01064...
3    s1_test_0003.wav  [0.01114464, 0.0063425223, 0.004875928, 0.0056...
4    s1_test_0004.wav  [0.0072339647, 0.008067443, 0.0077829952, 0

In [11]:
import pandas as pd
import numpy as np
from statistics import mean
import os

labels = {
    0: 'bird',
    1: 'eight',
    2: 'falcon',
    3: 'five',
    4: 'four',
    5: 'nine',
    6: 'one',
    7: 'seven',
    8: 'six',
    9: 'snake',
    10: 'three',
    11: 'two',
    12: 'zero'}

df1 = pd.read_csv('softmax_outputs.csv', names=['file', 'softmax'])
df2 = pd.read_csv('softmax_outputs2.csv', names=['file', 'softmax'])
df_comb = df1.copy()
df_comb['softmax2'] = df2['softmax']

i = 0
def avg_softmax(input1, input2):
    global i
    softmax1_list = list(input1)
    softmax2_list = list(input2)
    assert len(softmax1_list) == len(softmax2_list)
    softmax1 = softmax1_list[i].strip('[]').split()
    softmax2 = softmax2_list[i].strip('[]').split()
    softmax_avg = []
    for j in range(len(softmax1)):
        avg = mean([float(softmax1[j]), float(softmax2[j])])
        softmax_avg += [avg]
    i += 1
    return softmax_avg


def get_label(softmax):
    global j
    if max(softmax) < 0.5:
        print(labels[np.argmax(softmax)])
    return labels[np.argmax(softmax)]

df_avg = df_comb.apply(lambda avg: avg_softmax(df_comb['softmax'], df_comb['softmax2']), axis=1)
avg_pred = df_avg.map(lambda label: get_label(label))
filenames = map(lambda x: x.replace('_normalized', ''), os.listdir('s1_test_release_normalized/'))
df = pd.DataFrame(list(zip(filenames, avg_pred)))  # for ensemble
df.to_csv('challenge_2_team_Tensor is not flowing.csv', index=False, header=False)
print(df)

five
two
falcon
two
snake
zero
zero
six
eight
snake
six
falcon
snake
bird
eight
zero
falcon
six
                    0       1
0    s1_test_0000.wav   seven
1    s1_test_0001.wav  falcon
2    s1_test_0002.wav    four
3    s1_test_0003.wav   snake
4    s1_test_0004.wav    four
..                ...     ...
644  s1_test_0644.wav   seven
645  s1_test_0645.wav     one
646  s1_test_0646.wav    five
647  s1_test_0647.wav  falcon
648  s1_test_0648.wav     two

[649 rows x 2 columns]
